### Import libraries

In [24]:
import pandas as pd
import requests
import json
import csv

### Load the dataset

In [2]:
sales = pd.read_csv('sales.csv')

In [3]:
sales.head(5)

,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


### Find an API service

For this exercise, I'm using the [RAWG Video Games Database API](https://rawg.io/apidocs), which is the largest video game database and game discovery service. They offer various endpoints, but this time I'm focusing on their `games` endpoint to extract additional information for the games based on their title:

* Release date (`results.released`)
* Metacritic score (`results.metacritic`)

It is possible that not all games will have a match in the database, as searching on names isn't too fortunate because of e.g. differing capitalization, usage of special characters etc.

In [29]:
game_json = json.loads(requests.get('https://api.rawg.io/api/games').text)
game_json

{'count': 368120,
 'next': 'https://api.rawg.io/api/games?page=2',
 'previous': None,
 'results': [{'id': 3498,
   'slug': 'grand-theft-auto-v',
   'name': 'Grand Theft Auto V',
   'released': '2013-09-17',
   'tba': False,
   'background_image': 'https://media.rawg.io/media/games/b11/b115b2bc6a5957a917bc7601f4abdda2.jpg',
   'rating': 4.48,
   'rating_top': 5,
   'ratings': [{'id': 5,
     'title': 'exceptional',
     'count': 1889,
     'percent': 58.79},
    {'id': 4, 'title': 'recommended', 'count': 1081, 'percent': 33.64},
    {'id': 3, 'title': 'meh', 'count': 191, 'percent': 5.94},
    {'id': 1, 'title': 'skip', 'count': 52, 'percent': 1.62}],
   'ratings_count': 3183,
   'reviews_text_count': 17,
   'added': 10740,
   'added_by_status': {'yet': 202,
    'owned': 6871,
    'beaten': 2582,
    'toplay': 310,
    'dropped': 440,
    'playing': 335},
   'metacritic': 96,
   'playtime': 68,
   'suggestions_count': 413,
   'user_game': None,
   'reviews_count': 3213,
   'saturated_co

### Construct queries and pull information from API

* Create an array of unique games in my current dataset

In [45]:
games = sales['name'].unique()
len(games)
games_temp = games[:10]
games_temp

array(['Wii Sports', 'Super Mario Bros.', 'Mario Kart Wii',
       'Wii Sports Resort', 'Pokemon Red/Pokemon Blue', 'Tetris',
       'New Super Mario Bros.', 'Wii Play', 'New Super Mario Bros. Wii',
       'Duck Hunt'], dtype=object)

There are 11325 different games in my current `sales` dataset, we'll see how many we can match from the API.

* Connect to the API based on the constracted URLs and create a dictionary

In [42]:
game_api = 'https://api.rawg.io/api/games?name='
games_dict = {}
games_metadata_dict = {}

for game in games_temp:
    game_resp = json.loads(requests.get(game_api+game).text)['results']
    
    games_metadata_dict.update(
        {
            'release_date': game_resp[0]['released'],
            'metacritic_score': game_resp[0]['metacritic']
        }
    )
    
    games_dict.update({game: games_metadata_dict})

KeyboardInterrupt: 

I created a dictionary with the following layout:

```
{'name': 
    {
        'release_date',
        'metacritic_score'
    }
}
```

Where `name` is the name of the game from my original `sales` dataset, and `release_date` and `metacritic_score` are coming through the API calls.

### Save the dictionary

In [ ]:
with open('games.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in games_dict.items():
       writer.writerow([key, value])